In [38]:
import json
from bs4 import BeautifulSoup
import re
import datetime
from emoji import UNICODE_EMOJI
import itertools

import plotly.graph_objects as go
import scattertext as st
import nltk
import numpy as np
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import json
import scattertext as st
from nltk.tokenize import TweetTokenizer
import multidict as multidict

import os
from PIL import Image
from os import path
import matplotlib.pyplot as plt
import numpy as np

import random
from wordcloud import WordCloud, STOPWORDS
from stop_words import get_stop_words


In [39]:
stopwords = ["a","aby","aj","ale","anebo","ani","aniž","ano","asi","aspoň","atd","atp","az","ačkoli","až","bez","beze","blízko","bohužel","brzo","bude","budem","budeme","budes","budete","budeš","budou","budu","by","byl","byla","byli","bylo","byly","bys","byt","být","během","chce","chceme","chcete","chceš","chci","chtít","chtějí","chuti","ci","clanek","clanku","clanky","co","coz","což","cz","daleko","dalsi","další","den","deset","design","devatenáct","devět","dnes","do","dobrý","docela","dva","dvacet","dvanáct","dvě","dál","dále","děkovat","děkujeme","děkuji","email","ho","hodně","i","jak","jakmile","jako","jakož","jde","je","jeden","jedenáct","jedna","jedno","jednou","jedou","jeho","jehož","jej","jeji","jejich","její","jelikož","jemu","jen","jenom","jenž","jeste","jestli","jestliže","ještě","jež","ji","jich","jimi","jinak","jine","jiné","jiz","již","jsem","jses","jseš","jsi","jsme","jsou","jste","já","jí","jím","jíž","jšte","k","kam","každý","kde","kdo","kdy","kdyz","když","ke","kolik","kromě","ktera","ktere","kteri","kterou","ktery","která","které","který","kteři","kteří","ku","kvůli","ma","mají","mate","me","mezi","mi","mit","mne","mnou","mně","moc","mohl","mohou","moje","moji","možná","muj","musí","muze","my","má","málo","mám","máme","máte","máš","mé","mí","mít","mě","můj","může","na","nad","nade","nam","napiste","napište","naproti","nas","nasi","načež","naše","naši","ne","nebo","nebyl","nebyla","nebyli","nebyly","nechť","nedělají","nedělá","nedělám","neděláme","neděláte","neděláš","neg","nejsi","nejsou","nemají","nemáme","nemáte","neměl","neni","není","nestačí","nevadí","nez","než","nic","nich","nimi","nove","novy","nové","nový","nula","ná","nám","námi","nás","náš","ní","ním","ně","něco","nějak","někde","někdo","němu","němuž","o","od","ode","on","ona","oni","ono","ony","osm","osmnáct","pak","patnáct","po","pod","podle","pokud","potom","pouze","pozdě","pořád","prave","pravé","pred","pres","pri","pro","proc","prostě","prosím","proti","proto","protoze","protože","proč","prvni","první","práve","pta","pět","před","přede","přes","přese","při","přičemž","re","rovně","s","se","sedm","sedmnáct","si","sice","skoro","smí","smějí","snad","spolu","sta","sto","strana","sté","sve","svych","svym","svymi","své","svých","svým","svými","svůj","ta","tady","tak","take","takhle","taky","takze","také","takže","tam","tamhle","tamhleto","tamto","tato","te","tebe","tebou","ted'","tedy","tema","ten","tento","teto","ti","tim","timto","tipy","tisíc","tisíce","to","tobě","tohle","toho","tohoto","tom","tomto","tomu","tomuto","toto","trošku","tu","tuto","tvoje","tvá","tvé","tvůj","ty","tyto","téma","této","tím","tímto","tě","těm","těma","těmu","třeba","tři","třináct","u","určitě","uz","už","v","vam","vas","vase","vaše","vaši","ve","vedle","večer","vice","vlastně","vsak","vy","vám","vámi","vás","váš","více","však","všechen","všechno","všichni","vůbec","vždy","z","za","zatímco","zač","zda","zde","ze","zpet","zpravy","zprávy","zpět","čau","či", "čtrnáct","čtyři","šest","šestnáct","že"]


In [40]:
for sw in get_stop_words('cz'):
    if not (sw in stopwords):
        stopwords.append(sw)

## GET DATA

In [41]:
def read_file(filepath = None, text = None):
    if filepath:       
        with open(filepath, "r") as f:
            lines = f.readlines()
    elif text:
        lines = [text]
    else:
        raise Exception("Neither text or filepath was entered")

    soup = BeautifulSoup(lines[0], "html.parser")
    messages_to = soup.title.text
    names = [n.text for n in soup.findAll("div", {"class": "_3-96 _2pio _2lek _2lel"})]
    messages = [m.text for m in soup.findAll("div", {"class": "_3-96 _2let"})]
    times = [t.text for t in soup.findAll("div", {"class": "_3-94 _2lem"})]
    all_divs = [a.text for a in soup.findAll("div", {"class": "pam _3-95 _2pi0 _2lej uiBoxWhite noborder"})]

    names.reverse()
    times.reverse()
    messages.reverse()

    return list(zip(names, times, messages)), names, times, messages

def str_to_time(s_time):
    # date_time_str = '2018-06-29 08:15:27.243860'
    return datetime.datetime.strptime(s_time, '%d. %m. %Y %H:%M')

def get_emoji_reaction(m, w):
    if m.endswith(w):
        emoji_position = len(m) - len(w) - 2 
        emoji_ = m[emoji_position]
        if is_emoji(emoji_):
            return emoji_
    return None

def delete_reaction_end(m, writers):
    for w in writers:
        emoji = get_emoji_reaction(m,w)
        if emoji:
            return m[:len(m)-len(w)-2]
    return m

def replace_hidden_emoji(m):
    new_m = []
    laugh_pattern = '^.*(:[dD])+.*$'
    smile_pattern = "^.*(:\))+.*$"
    smile = re.compile(smile_pattern)
    laugh = re.compile(laugh_pattern)
    for w in m.split():
        laugh_match = laugh.search(w)
        smile_match = smile.search(w)
        new_word = w

        if laugh_match:
            splitted = w.split(":")
            if splitted[-1].lower() == "d":
                    new_word = new_word[:-2] + "😀"        
        elif smile_match:
            splitted = w.split(":")
            if splitted[-1].lower() == ")":
                    new_word = new_word[:-2] + "😊"

        new_m.append(new_word)
    return " ".join(new_m)

def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)

def is_emoji_in_message(message):
    for c in message:
        if is_emoji(c):
            return True
    return False

def extract_emojis(s):
    emojis = []
    for word in s:
        for c in word:
            if c in ["♂","♀"]:
                continue
            if is_emoji(c):
                emojis.append(c)
    return emojis

def get_writers_messages(messages, names):
    writers_messages = {}
    for i in range(len(messages)):
        writers_messages[names[i]] = writers_messages.get(names[i], []) + [messages[i]]
    return writers_messages


def get_writers_words(messages, names):
    writers_words = {}
    for i in range(len(messages)):
        message = messages[i]
        writer = names[i]
        words = messages[i].split()
        for word in words:
            writers_words[writer] = writers_words.get(writer, []) + [word.lower()]
    return writers_words

def is_reacted_message(m, writers):
    for w in writers:
        if m.endswith(w):
            emoji_position = len(m) - len(w) - 2 
            emoji_ = m[emoji_position]
            if is_emoji(emoji_):
                return emoji_
    return None


## Get data from file

In [42]:
def get_data(filepath = None, text = None, hide_authors = False):
    data, n, t, m = read_file(filepath, text)
    names, times, messages = [], [], []
    for i in range(len(data)):
        if t and n:
            names.append(n[i])
            times.append(t[i])
            messages.append(replace_hidden_emoji(m[i]))

    writers = list(set(names))
    messages_with_reactions = messages
    messages_without_reactions = [replace_hidden_emoji(delete_reaction_end(m, writers)) for m in messages]

    times_string = times
    times_datetime = [str_to_time(t) for t in times]

    return names, times, writers, messages, messages_without_reactions



In [43]:
filepath2 = "/home/vena/Documents/ai/fb_mess/data1/dominikhanke_thycdq67qw/message_1.html"
filepath1 = '/home/vena/Documents/ai/fb_mess/data1/annamickova_pwbfftazwa/message_1.html'
filepath3 = "/home/vena/Documents/ai/fb_mess/data1/tripvol3_kjltaezdrw/message_1.html"
filepath = '/home/vena/Documents/ai/fb_mess/data1/annamickova_pwbfftazwa/message_1.html'

names, times, writers, messages, messages_without_reactions = get_data(filepath)

In [44]:
def statistics(messages, names, writers):
    writers_words = get_writers_words(messages_without_reactions, names)
    writers_messages = get_writers_messages(messages_without_reactions, names)

    print("---Number of messages---")
    for writer in writers:
        print(f"{writer}: {len(writers_messages[writer])}")

    print("\n---Number of words---")
    for writer in writers:
        print(f"{writer}: {sum([len(mes.split()) for mes in writers_messages[writer]])}")

    print("\n---Number of characters---")
    for writer in writers:
        print(f"{writer}: {sum([len(mes) for mes in writers_messages[writer]])}")

    print("\n---Average message length (words)---")
    for writer in writers:
        print(f"{writer}: {round(sum([len(mes.split()) for mes in writers_messages[writer]]) / len(writers_messages[writer]), 2)}")

    print("\n---Longest message (words)---")
    for writer in writers:
        print(f"{writer}: {max([len(mes.split()) for mes in writers_messages[writer]])}")

    print("\n---Number of questions---")
    for writer in writers:
        print(f"{writer}: {len([mes for mes in writers_messages[writer] if ('?' in mes)])}")

In [45]:
def display_general_pies(writers, names, messages, nonames=False):
    writers_messages = get_writers_messages(messages, names)
    messages_count = [len(writers_messages[writers[i]]) for i in range(len(writers))]
    words_count = [sum([len(mes.split()) for mes in writers_messages[writers[i]]])for i in range(len(writers))]
    chars_count = [sum([len(mes) for mes in writers_messages[writers[i]]]) for i in range(len(writers))]
    questions_count = [len([mes for mes in writers_messages[writer] if ('?' in mes)]) for writer in writers]
    emojis_count = [len([mes for mes in writers_messages[writer] if (is_emoji_in_message(mes))]) for writer in writers]
    lenghty_count = [len([mes for mes in writers_messages[writer] if (len(mes.split()) > 10)]) for writer in writers]



    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]

    # colors = ["mediumturquoise", "darkorange"]
    fig = go.FigureWidget(make_subplots(rows = 2, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=['Počet zpráv', 'Počet slov', 'Počet znaků', "Zprávy s otázkou", "Zprávy s emoji", "Zprávy > 10 slov"]))
    
    trace0 = go.Pie(labels=writers, values=messages_count, name="Celkový počet zpráv")
    trace1 = go.Pie(labels=writers, values=words_count, name='Celkový počet slov')
    trace2 = go.Pie(labels=writers, values=chars_count, name="Celkový počet znaků")
    trace3 = go.Pie(labels=writers, values=questions_count, name="Celkový počet otázek")
    trace4 = go.Pie(labels=writers, values=emojis_count, name="Celkový počet emoji")
    trace5 = go.Pie(labels=writers, values=lenghty_count, name="Celkový počet dlouhých zpráv")

    fig.add_trace(trace0, 1, 1)
    fig.add_trace(trace1, 1, 2)
    fig.add_trace(trace2, 1, 3)
    fig.add_trace(trace3, 2, 1)
    fig.add_trace(trace4, 2, 2)
    fig.add_trace(trace5, 2, 3)


    fig.update_traces(hoverinfo="label+percent", textinfo="value", textfont_size=12, marker=dict(line=dict(color="black", width=2)))
    fig.update_layout(title_text = "Statistiky o počtu zpráv, slov a znaků", width=1200, height=700)
    
    # fig.show()
    
    return fig

# display_general_pies(writers, names, messages)

In [46]:
def display_messages_length_whisker_plots(messages, writers, names, nonames=False):
    writers_messages = get_writers_messages(messages, names)
    m_lengths = [[len(m) for m in writers_messages[writer]] for writer in writers]
    m_words = [[len(m.split()) for m in writers_messages[writer]] for writer in writers]

    fig = go.FigureWidget()

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]
    for i, writer in enumerate(writers):
        fig.add_trace(go.Violin(y=m_words[i],
                                name=writer,
                                box_visible=True,
                                meanline_visible=True,
                                points="all"))

    fig.update_layout(title="Rozložení počtu slov jednotlivých pisatelů", yaxis_title = "Poet slov", width = 1200, height=600)

    # fig.show()
    return fig

# display_messages_length_whisker_plots(messages_without_reactions, writers, names, True)    

## TIME THINGS

In [47]:
def get_timedeltas(names, times):
    time_series = {}
    last_writer, last_time = None, None

    for i in range(len(names)):
        respondent = names[i]
        m_time = str_to_time(times[i])

        if not last_writer:
            last_writer = respondent 
            last_time = m_time
            time_series[respondent] = time_series.get(respondent, []) + [0]
        else:
            time_delta = (m_time - last_time).total_seconds() / 60.0
            if respondent != last_writer:
                time_series[respondent] = time_series.get(respondent, []) + [time_delta]
                last_writer = respondent
                last_time = m_time
            else: # Not from the very last message 
                time_series[respondent] = time_series.get(respondent, []) + [-1]
                last_time = m_time
    return time_series

def get_avg_time_to_answear(writers, names, times):
    time_series = get_timedeltas(names, times)

    print("---Min time to answear---")
    min_times = {}
    for writer in writers:
        min_time = round(min(time_series[writer]) / 60, 1)
        min_times[writer] = min_time
        print(f"{writer}: {min_time} minutes")

    print("\n---Max time to answear---")
    max_times = {}
    for writer in writers:
        max_time = round(max(time_series[writer]) / 60, 1)
        max_times[writer] = max_time
        print(f"{writer}: {max_time} minutes")
    
    print("\n---Average time to answear---")
    avg_times = {}
    for writer in writers:
        avg_time = round(sum(time_series[writer]) / (len(time_series[writer]) * 60), 1)
        avg_times[writer] = avg_time
        print(f"{writer}: {avg_time} minutes")

    return avg_times

def get_immediate_responses_count(writers, names, times, immediate = 5):
    time_series = get_timedeltas(names, times)
    
    # print("\n---Immediate responses (less than 5 minute)---")
    zero_times = {}
    for writer in writers:
        zero_count = len([t for t in time_series[writer] if (t < immediate and t >= 0)])
        zero_times[writer] = zero_count
        # print(f"{writer}: {zero_count}")

    return zero_times

def get_long_responses_count(writers, names, times, immediate = 60):
    time_series = get_timedeltas(names, times)
    
    # print(f"\n---Immediate responses (less than {immediate} minutes)---")
    zero_times = {}
    for writer in writers:
        zero_count = len([t for t in time_series[writer] if (abs(t) > immediate)])
        zero_times[writer] = zero_count
        # print(f"{writer}: {zero_count}")

    return zero_times

def get_longest_message_streak(names, messages):
    last_name = None
    longest_streak = {}
    longest_streak_messages = {}
    starts_from = {}
    message_streak = []
    current_streak = 1
    for index, name in enumerate(names):
        if not last_name:
            last_name = name
        else:
            if last_name == name:
                current_streak += 1
                message_streak.append(messages[index])
            else:
                if longest_streak.get(last_name, 1) < current_streak:
                    longest_streak[last_name] = current_streak
                    longest_streak_messages[last_name] = message_streak

                current_streak = 1
                message_streak = []
            last_name = name
    
    return longest_streak_messages()

def get_conversations_starters(writers,names, times, threshold = 24 * 60):
    conv_started = {}
    for writer in writers:
        conv_started[writer] = 0
    last_writer, last_time = None, None

    for i in range(len(names)):
        respondent = names[i]
        m_time = str_to_time(times[i])

        if i == 0:
            conv_started[respondent] = conv_started.get(respondent, 0) + 1
        else:
            time_delta = (m_time - last_time).total_seconds() / 60.0
            if time_delta > threshold:
                conv_started[respondent] = conv_started.get(respondent, 0) + 1

        last_time = m_time

    return conv_started

def get_writers_timedeltas(i, writers):
    timedeltas = get_timedeltas(names, times)
    return [t for t in timedeltas[writers[i]] if t != -1]

def get_weekdays(writers, names, times):
    writers_weekdays = {}
    day_name= ['Pondělí', 'Úterý', 'Středa', 'Čtvrtek', 'Pátek', 'Sobota','Neděle']
    for i in range(len(names)):
        t = times[i]
        writer = names[i]
        writers_weekdays[writer] = writers_weekdays.get(writer, []) + [day_name[str_to_time(t).weekday()]]

    return writers_weekdays

def writer_messages_in_a_day(dates, writer, times, messages):
    dates_messages = [[] for i in dates]
    for index, time in enumerate(times):
        if names[index] != writer:
            continue
        date = str_to_time(time).date()
        message = messages[index]
        for i, d in enumerate(dates):
            if d == date:
                dates_messages[i] = dates_messages[i] + [message]
    return dates_messages


In [48]:
def display_conversations_pies(writers, names, times, messages, nonames=False):
    colors = ["mediumturquoise", "darkorange"]
    immediate_responses = [i for i in get_immediate_responses_count(writers, names, times, 5).values()]
    long_responses = [i for i in get_immediate_responses_count(writers, names, times, 2 * 60).values()]
    conv_starters = [i for i in get_conversations_starters(writers, names, times, 36 * 60).values()]

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]
    
    fig = go.FigureWidget(make_subplots(rows = 1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=['Do 5 minut', 'Do 2 hodin', "Starty konverzací"]))
    trace0 = go.Pie(labels=writers, values=immediate_responses, name="méně než 5 minut")
    trace1 = go.Pie(labels=writers, values=long_responses, name='méně jak 2 hodiny')
    trace2 = go.Pie(labels=writers, values=conv_starters, name='více jak 24 hodin')

    fig.add_trace(trace0, 1, 1)
    fig.add_trace(trace1, 1, 2)
    fig.add_trace(trace2, 1, 3)

    fig.update_traces(hoverinfo="label+percent", textinfo="value", textfont_size=15, marker=dict(line=dict(color="black", width=2)))
    fig.update_layout(title_text = "Rychlosti odpovědí", width=1200)
    # fig.show()
    return fig

# display_conversations_pies(writers, names, times, messages_without_reactions)

In [49]:
def display_answer_time_histogram(writers, names, times, nonames=False):
        timedeltas = get_timedeltas(names, times)
        hist_data = [get_writers_timedeltas(i, writers) for i in range(len(writers))]
        group_labels = writers
        colors = ["#393E46", "#F66095"]

        if nonames:
                writers = ["Author_" + f"{i}" for i in range(len(writers))]
        fig = go.FigureWidget()
        for i in range(len(writers)):
                fig.add_trace(go.Histogram(histfunc="count", x = hist_data[i], name=writers[i], xbins=dict(
                        start=0.0,
                        end=30.0,
                        size=1)))

        fig.update_layout(barmode="overlay",title="Jak rychle odpovídám? (0 až 30 min)", xaxis_title='Doba odpovědi (min)', yaxis_title='Počet odpovědí', width=1200)
        fig.update_traces(opacity=0.5)

        # fig.update_traces(hoverinfo="label")
        # fig.show()
        return fig

# display_answer_time_histogram(writers, names, times)

In [50]:
def display_messages_in_weekdays_histogram(writers, names, times, nonames=False):
    day_name= ['Pondělí', 'Úterý', 'Středa', 'Čtvrtek', 'Pátek', 'Sobota','Neděle']
    writers_weekdays = get_weekdays(writers, names, times)
    hist_data = [writers_weekdays[writers[i]] for i in range(len(writers))]
    
    for i, w in enumerate(writers):
        days = hist_data[i]
        new_days = []
        weekdays_counter = [0] * len(day_name)
        for d in days:
            for k, weekday in enumerate(day_name):
                if d == weekday:
                    weekdays_counter[k] += 1

        for j, counter in enumerate(weekdays_counter):
            new_days = new_days + [day_name[j] for _ in range(counter)]
        hist_data[i] = new_days

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]

    fig = go.FigureWidget()
    for i in range(len(writers)):
        fig.add_trace(go.Histogram(histfunc="count", x = hist_data[i], name=writers[i]))

    fig.update_layout(barmode="stack",title="Agregovaný počet zpráv ve dnech", xaxis_title='Den v týdnu', yaxis_title='Počet zpráv', width=1000)
    # fig.update_traces(hoverinfo="label")
    # fig.show()
    return fig

# display_messages_in_weekdays_histogram(writers, names, times)

In [51]:
def display_half_hours_histogram(writers, names, times, nonames=False):
    hours = [str_to_time(t).hour + (str_to_time(t).minute / 60) for t in times]
    writers_hours = [[] for writer in writers]

    for i, h in enumerate(hours):
        writer = names[i]
        for j, ww in enumerate(writers):
            if ww == writer:
                writers_hours[j].append(h)

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]

    fig = go.FigureWidget()
    for i in range(len(writers)):
        fig.add_trace(go.Histogram(x=writers_hours[i], xbins=dict(start=0, end=24, size=1), name=writers[i]))

    fig.update_layout(barmode="stack", title_text = "Agregovaný počet zpráv v půlhodinových intervalech", xaxis_title = "Hodiny", yaxis_title = "Počet zpráv", width = 1000)

    # fig.update_traces(opacity=0.75)
    # fig.show()
    return fig

# display_half_hours_histogram(writers, names, times)


In [58]:
def display_messages_in_time_scatter(writers, names, times, messages, nonames=False):
    dates = sorted(list(set([str_to_time(t).date() for t in times])))
    day_messages = {}
    len_day_messages = {}
    for writer in writers:
        day_messages[writer] = writer_messages_in_a_day(dates, writer, times, messages)
        len_day_messages[writer] = [len(ms) for ms in day_messages[writer]]

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]
        new_len_day = {}
        for i, w in enumerate(list(len_day_messages.keys())):
            new_len_day[writers[i]] = len_day_messages[w]
        len_day_messages = new_len_day

    dates = ["{}-{}-{}".format(o.year, o.month, o.day) for o in dates]
            
    
    fig = go.FigureWidget()
    for i in range(len(writers)):
        fig.add_trace(go.Scatter(x=dates, y=len_day_messages[writers[i]], name=writers[i]))
    # fig.add_trace(go.Scatter(x=dates, y=len_day_messages[writers[1]], name=writers[1]))

    fig.update_traces(mode="markers+lines", marker_size=10, marker_line_width=2)
    fig.update_layout(title="Počet zpráv v čase", yaxis_zeroline=False, xaxis_zeroline=False, width=1200)

    # fig.show()
    return fig

# display_messages_in_time_scatter(writers, names, times, messages)
# 

FigureWidget({
    'data': [{'marker': {'line': {'width': 2}, 'size': 10},
              'mode': 'markers+line…

## WORD THINGS

In [206]:
def get_chars(words_list):
    chars = {}
    for word in words_list:
        for c in word:
            chars[c] = chars.get(c, 0) + 1
    return chars
    
def get_bow(words_list):
    bag_of_words = {}
    for line in words_list:
        if line in bag_of_words:
            bag_of_words[line] += 1
        else:
            bag_of_words[line] = 1
    
    return dict(sorted(bag_of_words.items(), key=lambda item: item[1], reverse=True))

def get_n_grams(words_list, n=2):
    buffer = []
    n_grams = {}
    for i, word in enumerate(words_list):
        buffer.append(word)

        if len(buffer) == n:
            n_gram_tuple = tuple(buffer)
            buffer = [buffer[-1]]

            if n_gram_tuple in n_grams:
                n_grams[n_gram_tuple] += 1
            else:
                n_grams[n_gram_tuple] = 1
    return n_grams  

def get_messages_containing(messages, word):
    ms = []
    for m in messages:
        for w in m.split():
            if w.lower() == word.lower():
                ms.append(m)
    return ms

def get_most_n_grams(writers, messages, names, stopwords = [], count = 10, n=2):
    writers_words = get_writers_words(messages, names)
    most_used_ngrams = {}
    for writer in writers:
        words = writers_words[writer]
        if stopwords:
            words = [word for word in writers_words[writer] if word not in stopwords]
        

        my_words = get_n_grams(words, n=n)
        sorted_words = dict(sorted(my_words.items(), key=lambda item: item[1], reverse=True))
        sliced_words = dict(itertools.islice(sorted_words.items(), count))

        most_used_ngrams[writer] = sliced_words

    return most_used_ngrams

def most_used_words(writers, messages, names, stopwords = [], count = 10):
    writers_words = get_writers_words(messages, names)
    most_used_words = {}
    for writer in writers:
        words = writers_words[writer]
        if stopwords:
            words = [word for word in writers_words[writer] if word not in stopwords]
        

        my_words = get_bow(words)
        sorted_words = dict(sorted(my_words.items(), key=lambda item: item[1], reverse=True))
        sliced_words = dict(itertools.islice(sorted_words.items(), count))

        most_used_words[writer] = sliced_words

    return most_used_words

def is_emoji_word(word):
    for c in word:
        if is_emoji(c):
            return True
    return False
    
def get_freq_dict(writers, messages, names, stopwords = []):
    writers_words = get_writers_words(messages, names)
    most_used_words = {}
    words = []
    for writer in writers:
        new_words = writers_words[writer]
        new_words = [word for word in new_words if not is_emoji_word(word)]

        if stopwords:
            words = [word for word in new_words if word not in stopwords]
        words += new_words
        

    my_words = get_bow(words)
    sorted_words = dict(sorted(my_words.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_words


In [5]:
# get_most_n_grams(writers,messages_without_reactions, names, n=3)

In [6]:
# most_used_words(writers, messages_without_reactions, names, stopwords)

### ScatterText

In [40]:
def create_scatter_text(writers, names, messages, nonames=False):
    my_df = pd.DataFrame({"author": names, "message": messages})
    nlp = st.tweet_tokenizier_factory(nltk.tokenize.TweetTokenizer())
    my_df['parse'] = my_df['message'].apply(nlp)

    # df = st.SampleCorpora.ConventionData2012.get_data().assign(
    #     parse=lambda df: df.text.apply(st.whitespace_nlp_with_sentences)
    # )

    corpus = st.CorpusFromParsedDocuments(
        my_df, category_col='author', parsed_col='parse'
    ).build().get_unigram_corpus().compact(st.AssociationCompactor(2000))

    if nonames:
        html = st.produce_scattertext_explorer(
        corpus,
        category=writers[0], category_name="Author_0", not_category_name="Author_1",
        minimum_term_frequency=0, pmi_threshold_coefficient=0,
        width_in_pixels=1000,# metadata=corpus.get_df()['speaker'],
        transform=st.Scalers.dense_rank
    )
    else:
        html = st.produce_scattertext_explorer(
            corpus,
            category=writers[0], category_name=writers[0], not_category_name=writers[1],
            minimum_term_frequency=0, pmi_threshold_coefficient=0,
            width_in_pixels=1000,# metadata=corpus.get_df()['speaker'],
            transform=st.Scalers.dense_rank
        )

    with open('./demo_compact.html', 'w') as f:
        f.write(html)
    f.close()

### WordCloud

In [7]:
def getFrequencyDictForText(sentence):
    fullTermsDict = multidict.MultiDict()
    tmpDict = {}

    # making dict for counting frequencies
    for text in sentence.split(" "):
        val = tmpDict.get(text, 0)
        tmpDict[text.lower()] = val + 1
    for key in tmpDict:
        fullTermsDict.add(key, tmpDict[key])
    return fullTermsDict


def makeImage(text, image_file):
    alice_mask = np.array(Image.open(image_file))

    wc = WordCloud(background_color="white", max_words=1000, mask=alice_mask)
    # generate word cloud
    wc.generate_from_frequencies(text)
    wc.to_file("used_words.png")
    # show
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()


def create_word_cloud(writers, messages, names, image_file):
    # get data directory (using getcwd() is needed to support running example in generated IPython notebook)
    d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

    makeImage(get_freq_dict(writers, messages, names, stopwords), image_file)

# create_word_cloud(writers, messages_without_reactions, names, "fb_logo.png")

In [135]:
def grey_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

def custom_word_cloud(messages):
    # get data directory (using getcwd() is needed to support running example in generated IPython notebook)
    d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

    # read the mask image taken from
    # http://www.stencilry.org/stencils/movies/star%20wars/storm-trooper.gif
    mask = np.array(Image.open(path.join(d, "fb_logo.png")))

    text = []
    for m in messages:
        text.append(m)
    text = " ".join(text)
    # print(type(text))

    # adding movie script specific stopwords
    stopwords = set(STOPWORDS)
    stopwords.add("int")
    stopwords.add("ext")

    wc = WordCloud(max_words=1000, mask=mask, stopwords=stopwords, margin=10,
                random_state=1).generate(text)
    # store default colored image
    default_colors = wc.to_array()
    plt.title("Custom colors")
    plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3),
            interpolation="bilinear")
    wc.to_file("a_new_hope.png")
    plt.axis("off")
    plt.figure()
    plt.title("Default colors")
    plt.imshow(default_colors, interpolation="bilinear")
    plt.axis("off")
    plt.show()

## EMOJIS

In [133]:
def get_emoji_reactions(messages):
    emoji_reactions = {}
    for message in messages:
        for w in writers:
            emoji = get_emoji_reaction(message, w)
            if emoji:
                emoji_reactions[w] = emoji_reactions.get(w, []) + [emoji]
    return emoji_reactions


def get_messages_which_was_reacted_to(writers, names, messages, w):
    reacted_messages = []
    reactions = []
    reactors = []
    for i, m in enumerate(messages):
        if is_reacted_message(m, writers) and names[i] == w:
            reacted_messages.append(replace_hidden_emoji(delete_reaction_end(m, writers)))
            for writer in writers:
                emoji = get_emoji_reaction(m, writer)
                if emoji:
                    reactions.append(emoji)
                    reactors.append(writer)
                    break
    return list(zip(reacted_messages, reactions, reactors))     

def get_writers_emojis(writers, messages):
    writers_emojis = {}
    for i, m in enumerate(messages):
        without_reaction = delete_reaction_end(m, writers)
        replaced_emojis = replace_hidden_emoji(without_reaction)
        emojis = extract_emojis(replaced_emojis.split())
        if emojis:
            writers_emojis[names[i]] = writers_emojis.get(names[i], []) + [emojis]
    return writers_emojis

def get_used_emojis(writers, messages):    
    written_emojis = get_writers_emojis(writers, messages)
    used_emojis = {}
    for writer in writers:
        used_emojis[writer] = {}
        if writer in written_emojis.keys():
            for e in written_emojis[writer]:
                for emoji_list in e:
                    for emoji in emoji_list:
                        used_emojis[writer][emoji] = used_emojis[writer].get(emoji, 0) + 1
    return  used_emojis

def get_reactions_to_messages(writers, names, messages):
    reactions = {}
    for writer in writers:
        reactions[writer] = get_messages_which_was_reacted_to(writers, names, messages, writer)
    return reactions

def get_emoji_reaction_count(writers, messages):
    emoji_reactions = {}
    reactions = get_reactions_to_messages(writers, names, messages)
    for writer in writers:
        emoji_reactions[writer] = {}
        for r in reactions[writer]:
            reaction = r[1]
            emoji_reactions[writer][reaction] = emoji_reactions[writer].get(reaction, 0) + 1
    return emoji_reactions 

In [32]:
def display_emoji_histogram(writers, messages, nonames = False):
    used_emojis = get_used_emojis(writers, messages)

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]
        new_used_emoji = {}
        for i, w in enumerate(list(used_emojis.keys())):
            new_used_emoji[writers[i]] = used_emojis[w]
        used_emojis = new_used_emoji

    fig = go.FigureWidget()
    for i in range(len(writers)):
        fig.add_trace(go.Histogram(histfunc="sum", y=list(used_emojis[writers[i]].values()), x=list(used_emojis[writers[i]].keys()), name=writers[i]))

    fig.update_layout(title="Jaké emoji používám?", xaxis_title='Emoji', yaxis_title='Počet', width=1200)
    # fig.show()
    return fig

# display_emoji_histogram(writers, messages_without_reactions, True)

In [33]:
def display_emoji_reactions(writers, messages, nonames=False):
    emoji_reactions = get_emoji_reaction_count(writers, messages)

    if nonames:
        writers = ["Author_" + f"{i}" for i in range(len(writers))]
        new_emoji_react = {}
        for i, w in enumerate(list(emoji_reactions.keys())):
            new_emoji_react[writers[i]] = emoji_reactions[w]
        emoji_reactions = new_emoji_react
            
    fig = go.FigureWidget()
    for i in range(len(writers)):
        fig.add_trace(go.Histogram(histfunc="sum", y=list(emoji_reactions[writers[i]].values()), x=list(emoji_reactions[writers[i]].keys()), name=writers[i]))

    fig.update_layout(title="Jaké reakce jsem dostal?", xaxis_title='Emoji', yaxis_title='Počet', width=1000)
    # fig.show()

    return fig

# display_emoji_reactions(writers, messages, True)